# Custom Vision: Hochladen von gelabelten Bildern mit Python

In [1]:
ENDPOINT = "https://eastus.api.cognitive.microsoft.com/"

# Replace with a valid key
# You can find the keys in the CustomVision property
training_key = "f27bc63d4a8944e687627072687a73c3"
project_name = "WaterMeterDetection (2)"
project_id = "fbd892e5-7020-453b-b7fa-1ffc1da2c662"

In [52]:
from ipywidgets import Button,Text, Layout


from IPython.display import display
training_key = widgets.Text(placeholder="Custom Vison Trainings Key")
display(training_key)

endpoint = widgets.Text(placeholder="Endpoint")
display(training_key)


project_name = widgets.Text(description="CustomVison Projekt Name", layout=b)
display(project_name)

project_id = widgets.Text(description="Projekt ID")
display(project_id)

button = Button(description="Bilder Hochladen", layout=b)
display(button)


def handle_submit(sender):
    print("test")
    
button.on_click(handle_submit)

Text(value='', placeholder='Custom Vison Trainings Key')

Text(value='', placeholder='Custom Vison Trainings Key')

Text(value='', description='CustomVison Projekt Name', layout=Layout(width='auto'))

Text(value='', description='Projekt ID')

Button(description='Bilder Hochladen', layout=Layout(width='auto'), style=ButtonStyle())

In [20]:
# Imports
import os
import pandas as pd

try:
    from azure.cognitiveservices.vision.computervision import ComputerVisionClient
    from azure.cognitiveservices.vision.computervision.models import ImageFileCreateEntry, Region
except:
    !pip install azure.cognitiveservices.vision.computervision
    from azure.cognitiveservices.vision.computervision import ComputerVisionClient
    from azure.cognitiveservices.vision.computervision.models import ImageFileCreateEntry, Region


  Using cached https://files.pythonhosted.org/packages/55/d8/f61816761877cdd9c0e1e2274914946e67b3a07dd59bda37636badcedba1/azure_cognitiveservices_vision_computervision-0.5.0-py2.py3-none-any.whl
You should consider upgrading via the 'pip install --upgrade pip' command.


ImportError: cannot import name 'ImageFileCreateEntry'

In [22]:

trainer = CustomVisionTrainingClient(training_key, endpoint=ENDPOINT)

# Create a new project
print ("Connect to project...")
project = trainer.get_project(project_id, custom_headers=None, raw=False)
print(project.id)

try:
    # Create a new tag
    tag_str = "NumberArea"
    NumberArea_tag = trainer.create_tag(project.id, tag_str)
except:
    pass

#! Upload Images
# Update this with the path to where you downloaded the images.
train_images_path = "/home/nbuser/library/data/train_images/"
label_path = "/home/nbuser/library/data/labels/image_regions.csv"
train_file_names = os.listdir(train_images_path)
labels_df = pd.read_csv(label_path)

# Go through the data table above and create the images
print("Adding images...")

#! Image need to uploaded in batches with max size of 64 samples
# function to create the batches
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

# use the batch function to create batches
batch_num = 0
for batch_samples in batch(train_file_names, 64):
    batch_num =+ 1
    print("Starting Batch: " + str(batch_num) + " / " + str((len(train_file_names)/64 )+1))
    print(len(batch_samples))

    tagged_images_with_regions = []

    # iterate throw all batches
    for file_name in batch_samples:
        # select the region labels from the image_redions.csv by the image_id
        row = labels_df[labels_df["image_id"]==file_name]
        
        # save the labels from the dataframe into variables
        x,y,w,h =  row["left"],row["top"], row["width"],  row["height"]
        image_id = row["image_id"]

        # create the CustomVision Region objects 
        regions = [Region(tag_id=NumberArea_tag.id, left=x,top=y,width=w,height=h)]

        # load the images and combine the image data withe the
        with open(train_images_path + file_name, mode="rb") as image_contents:
            tagged_images_with_regions.append(ImageFileCreateEntry(name=image_id, contents=image_contents.read(), regions=regions))

    upload_result = trainer.create_images_from_files(project.id, images=tagged_images_with_regions)

    if not upload_result.is_batch_successful:
        print("Image batch upload failed.")
        for image in upload_result.images:
            print("Image status: ", image.status)


Connect to project...
fbd892e5-7020-453b-b7fa-1ffc1da2c662
Adding images...
Starting Batch: 1 / 2.421875
64
Starting Batch: 1 / 2.421875
27
Image batch upload failed.
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OKDuplicate
